In [1]:
import sys
sys.path.append('/workspace/nabang1010/STEAM/LeGiaBach_STEAM/src/yolov5')
import torch
import os
import platform
import sys
from pathlib import Path
import numpy as np
import pandas as pd
from PIL import Image, ImageOps

from models.common import DetectMultiBackend
from utils.dataloaders import IMG_FORMATS, VID_FORMATS, LoadImages, LoadScreenshots, LoadStreams
from utils.general import (LOGGER, Profile, check_file, check_img_size, check_imshow, check_requirements, colorstr, cv2,
                           increment_path, non_max_suppression, print_args, scale_boxes, strip_optimizer, xyxy2xywh)
from utils.plots import Annotator, colors, save_one_box
from utils.torch_utils import select_device, smart_inference_mode


/root/anaconda3/envs/DATN/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device  = select_device('0')
device

YOLOv5 🚀 v6.2-177-g2373d54 Python-3.7.13 torch-1.12.1+cu102 CUDA:0 (Quadro RTX 5000, 16117MiB)



device(type='cuda', index=0)

In [3]:
weights = "/workspace/nabang1010/STEAM/LeGiaBach_STEAM/src/yolov5/runs/train/yolov5x_hyp4/weights/last.pt"
data =  "/workspace/nabang1010/STEAM/LeGiaBach_STEAM/src/yolov5/chestxray16.yaml"


In [4]:
model = DetectMultiBackend(weights, device=device, data=data)
model.eval()

Fusing layers... 
YOLOv5x summary: 322 layers, 86274349 parameters, 0 gradients, 204.1 GFLOPs


DetectMultiBackend(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 80, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(80, 160, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (act): SiLU(inplace=True)
      )
      (2): C3(
        (cv1): Conv(
          (conv): Conv2d(160, 80, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(160, 80, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (cv3): Conv(
          (conv): Conv2d(160, 160, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (m): Sequential(
          (0): Bottleneck(
            (cv1): Conv(
              (conv): Conv2d(80, 80, kernel_size=(1, 1), stride=(1, 1))
              (act): SiLU(inplace=True)
            )
            (cv2): 

In [5]:
DF_RAW = pd.read_csv("/workspace/nabang1010/STEAM/LeGiaBach_STEAM/src/EfficientNetB4/submission.csv")
DF_RAW.head()

,image_id,abnormal
0,26a5285339df294ce288663b092c23c3,1
1,31ccf1b8a9992b055d0d26043eea3c30,1
2,688ecdb1a4e994d42b5a50a8c4a9736f,1
3,73a47bff27e977dc25c0b2cd4a470eea,1
4,8bb8636f45e172949bb99b3d5dd715fd,1


In [6]:
stride, names, pt = model.stride, model.names, model.pt
vid_stride = 1


In [37]:
source = "/workspace/nabang1010/STEAM/LeGiaBach_STEAM/DATA/dataset/test_png"

img_size = (640, 640)
dataset = LoadImages(source, img_size=img_size, stride=stride, auto=pt, vid_stride=vid_stride)
DF_submissions = pd.DataFrame(columns=['ID', 'class_id', 'label', 'x_min', 'x_max', 'ymin', 'ymax', 'score'])
for path, im, im0s, vid_cap, s in  dataset:
    # print("=====================")
    img = torch.from_numpy(im).to(device).float()
    img /= 255.0
    if img.ndimension() == 3:
        img = img[None]
    pred = model(img, augment=False)
    pred = non_max_suppression(pred, 0.4, 0.3, classes=None, agnostic=False)
    # print(path)
    # print image name
    image_name = path.split("/")[-1][0:-4]
    # print(image_name)
    for i, det in enumerate(pred):
        if len(det):
            p, im0, frame = path, im0s.copy(), getattr(dataset, 'frame', 0)
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]] 
            det[:, :4] = scale_boxes(img.shape[2:], det[:, :4], im0.shape).round()
            for *xyxy, conf, cls in reversed(det):
                # xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()
                label = f'{names[int(cls)]} {conf:.2f}'
                class_id = int(cls)
                # convert to coordinates in original image
                x1 = int(xyxy[0].item())
                y1 = int(xyxy[1].item())
                x2 = int(xyxy[2].item())
                y2 = int(xyxy[3].item())
                # conf to string
                conf = str(conf.item())
                # print(image_name, class_id, label, x1, x2, y1, y2, conf)
            
                DF_submissions = DF_submissions.append({'ID': str(image_name), 'class_id': str(class_id), 'label': str(label), 'x_min': str(x1), 'x_max': str(x2), 'ymin': str(y1), 'ymax': str(y2), 'score': conf}, ignore_index=True) 
        
DF_submissions.to_csv("/workspace/nabang1010/STEAM/LeGiaBach_STEAM/src/yolov5/submission_yolov5_15.csv", index=False)

In [38]:

DF_submissions

,ID,class_id,label,x_min,x_max,ymin,ymax,score
0,002a34c58c5b758217ed1f584ccbcfe9,14,No finding 0.99,0,2345,0,2584,0.9929428696632385
1,004f33259ee4aef671c2b95d54e4be68,14,No finding 0.99,0,2517,0,3027,0.9927288889884949
2,008bdde2af2462e86fd373a445d0f4cd,0,Aortic enlargement 0.84,1440,1736,824,1195,0.8434820771217346
3,008bdde2af2462e86fd373a445d0f4cd,3,Cardiomegaly 0.93,1102,1933,1412,1787,0.9344973564147949
4,008bdde2af2462e86fd373a445d0f4cd,15,Finding 0.99,0,2304,0,2880,0.9896560907363892
...,...,...,...,...,...,...,...,...
4702,ffc441e0c8b7153844047483a577e7c3,14,No finding 0.99,0,1944,0,2328,0.9927563071250916
4703,ffccf1709d0081d122a1d1f9edbefdf1,11,Pleural thickening 0.49,2461,2611,2559,2801,0.4945197105407715
4704,ffccf1709d0081d122a1d1f9edbefdf1,13,Pulmonary fibrosis 0.67,1564,2265,660,1626,0.674721360206604
4705,ffccf1709d0081d122a1d1f9edbefdf1,13,Pulmonary fibrosis 0.74,578,1045,726,1372,0.7449044585227966
